In [64]:
import pandas as pd
import polars as pl
import sqlalchemy as sa
import urllib
from datetime import datetime, timedelta
import math
import numpy as np
import datetime as dt
import glob
import numpy as np
from openpyxl import load_workbook
from openpyxl.styles import Alignment
import openpyxl
from openpyxl.styles import Alignment, Font, PatternFill
import pyodbc
from sqlalchemy import create_engine

In [65]:
### Connect to Azure server
azr_server = "ssd-synapse-prod.sql.azuresynapse.net"
azr_database = 'frtssdsqlpoolprod'
azr_username = 'frtcid'
azr_password = 'Frt2022Cid@#123'
connection_url = sa.engine.URL.create(
    "mssql+pyodbc",
    username=azr_username,
    password=azr_password,
    host=azr_server,
    database=azr_database,
    query={
        "driver": "ODBC Driver 17 for SQL Server",
        "autocommit": "True",
    },
)
azr_engine = sa.create_engine(connection_url)

In [66]:
sql_query = """
	SELECT 
		b.PRODUCT_ID,a.PRODUCT_CODE,a.PRODUCT_CATEGORY_NAME,a.FormMapName,
		b.GROUP_NAME_LV1,B.ATTRIBUTE_NAME ,c.W_INSERT_DT, string_agg(b.ATTRIBUTE_VALUE,', ') AS ATTRIBUTE_VALUE_ADD
	FROM 
		dbo.VD_CID_PRODUCT_ATTRIBUTE_HEADER_ICT a 
		LEFT JOIN dbo.VD_CID_PRODUCT_ATTRIBUTE_DETAIL_ICT b 
		ON a.PRODUCT_ID=b.PRODUCT_ID 
		LEFT JOIN dbo.VD_CID_PRODUCT C 
		ON A.PRODUCT_CODE=C.PRODUCT_CODE AND C.W_DATASOURCE_ID='ICT'
	WHERE
	a.PRODUCT_CATEGORY_NAME IN (N'Thiết bị gia dụng',N'Điện máy')
	AND a.FormMapName NOT IN ('Voucher - VPKM',N'Đồ dùng sinh hoạt',
								N'Vệ sinh nhà cửa',N'PK Khẩu trang lọc khí',N'PK - PKNK khác',N'Khẩu trang lọc khí',N'Khóa điện tử',N'Đồ dùng bếp
								',N'Đồ dùng sinh hoạt',N'Bình đun nước',N'Bình nước
								',N'Dụng cụ giữ nhiệt',N'Balo - Túi xách')
								AND C.PRODUCT_NAME NOT LIKE '%Demo%'
	GROUP BY b.PRODUCT_ID,a.PRODUCT_CODE,a.PRODUCT_CATEGORY_NAME,a.FormMapName,b.GROUP_NAME_LV1,B.ATTRIBUTE_NAME ,c.W_INSERT_DT
								ORDER BY C.W_INSERT_DT DESC, a.PRODUCT_CODE,b.ATTRIBUTE_NAME 
"""



In [67]:

df = pd.read_sql_query(sql_query, azr_engine)
df

,PRODUCT_ID,PRODUCT_CODE,PRODUCT_CATEGORY_NAME,FormMapName,GROUP_NAME_LV1,ATTRIBUTE_NAME,ATTRIBUTE_VALUE,W_INSERT_DT
0,36175,00900643,Thiết bị gia dụng,Nồi cơm điện,Thông số cơ bản,Loại hàng Ecom,Nồi nắp rời,2023-08-27 04:03:39.487
1,36175,00900643,Thiết bị gia dụng,Nồi cơm điện,Thiết kế & Trọng lượng,Chiều rộng kiện hàng (cm),30.2,2023-08-27 04:03:39.487
2,36175,00900643,Thiết bị gia dụng,Nồi cơm điện,Thông tin hàng hóa,Thời điểm ra mắt,2017,2023-08-27 04:03:39.487
3,36175,00900643,Thiết bị gia dụng,Nồi cơm điện,Thông tin hàng hóa,Sản phẩm có tính tồn kho,True,2023-08-27 04:03:39.487
4,36175,00900643,Thiết bị gia dụng,Nồi cơm điện,Thiết kế & Trọng lượng,Chiều rộng kiện hàng (cm),30.2,2023-08-27 04:03:39.487
...,...,...,...,...,...,...,...,...
149814,27562,00794151,Thiết bị gia dụng,Nồi cơm điện,Thông số cơ bản,Công suất,500 W,2022-01-27 04:32:09.570
149815,27562,00794151,Thiết bị gia dụng,Nồi cơm điện,Thông tin hàng hóa,Loại hàng,Nồi cơm điện,2022-01-27 04:32:09.570
149816,27562,00794151,Thiết bị gia dụng,Nồi cơm điện,Thông số cơ bản,Độ dày lòng nồi,1.030,2022-01-27 04:32:09.570
149817,27562,00794151,Thiết bị gia dụng,Nồi cơm điện,Thông tin hàng hóa,Sản phẩm PreOrder,True,2022-01-27 04:32:09.570


In [37]:
new_column_names = {'PRODUCT_CODE': 'Mã sản phẩm','PRODUCT_CATEGORY_NAME': 'Ngành hàng', 'FormMapName': 'Loại hàng','GROUP_NAME_LV1':'NHÓM THUỘC TÍNH','ATTRIBUTE_NAME':'THUỘC TÍNH','W_INSERT_DT':'Ngày cập nhật'}
df.rename(columns=new_column_names, inplace=True)

In [38]:
unique_loai_hang = df['Loại hàng'].unique()
writer = pd.ExcelWriter('/FRT/38.BackUp_Data/Output_pivot_Dai.xlsx', engine='xlsxwriter')

In [39]:
for loai in unique_loai_hang:
    sheet_name = loai
    df_loai_hang = df[df['Loại hàng'] == loai]
    privot_loaihang =df_loai_hang.pivot_table(index=['Mã sản phẩm','Ngành hàng','Loại hàng','Ngày cập nhật'], columns=['NHÓM THUỘC TÍNH','THUỘC TÍNH'], values='ATTRIBUTE_VALUE', aggfunc='first')
    privot_loaihang.to_excel(writer, sheet_name=sheet_name)
writer.save()

C:\Users\123\AppData\Local\Temp\ipykernel_45184\1415529767.py:6: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()


In [44]:
wb = load_workbook('/FRT/38.BackUp_Data/Output_pivot_Dai.xlsx')
for sheet in wb.sheetnames:
    ws = wb[sheet]
    e1_value = ws['D1'].value
    e2_value = ws['D2'].value
    ws.merge_cells(start_row=1, start_column=1, end_row=1, end_column=4)
    ws.merge_cells(start_row=2, start_column=1, end_row=2, end_column=4)
    merged_cell = ws.cell(row=1, column=1, value=e1_value).font = Font(bold=True)
    merged_cell = ws.cell(row=2, column=1, value=e2_value).font = Font(bold=True)
    merged_cell = ws.cell(row=1, column=1)
    merged_cell2 = ws.cell(row=2, column=1)
    merged_cell.alignment = Alignment(horizontal='center')
    merged_cell2.alignment = Alignment(horizontal='center')
    for column_cells in ws.columns:
        max_length = 0
        for cell in column_cells:
            try:
                if len(str(cell.value)) > max_length:
                    max_length = len(str(cell.value))
            except:
                pass
        adjusted_width = (max_length + 2)
        column_letter = openpyxl.utils.get_column_letter(column_cells[0].column)
        ws.column_dimensions[column_letter].width = adjusted_width
    ws['A1'].fill = PatternFill(start_color="FFFF66", end_color="FFFF66", fill_type="solid")
    ws['A2'].fill = PatternFill(start_color="CCFFFF", end_color="CCFFFF", fill_type="solid")
    for row in ws.iter_rows(min_row=3, max_row=3, min_col=1, max_col=4):
        for cell in row:
            cell.fill = PatternFill(start_color="FFCC66", end_color="FFCC66", fill_type="solid")
    for row_num in range(1, 3):
        ws.row_dimensions[row_num].height = 30
wb.save('/FRT/38.BackUp_Data/Output_pivot_temp_6.xlsx')